# Topic Modeling and Gibbs Sampling

Задача: описать текст через распределение весов по некоторому фиксированному набору топиков (тегов). Например, для набора тегов Политика, Военные сражения, Спорт, Интернет, Драма представить роман "Война и мир" как вектор (0.3, 0.2, 0, 0, 0.5), а статью в газете про допинг в велоспорте как вектор (0.1, 0, 0.7, 0, 0.2).

Для чего, например, это нужно: имея векторное представление для текстов, тексты можно сравнивать, рекомендовать похожие.

Условие: даны только набор текстов и количество тем.

## Немного теории

Будем представлять текст как неупорядоченный набор слов (Bag-of-words model). Предположим, что имеется K тегов и для каждого тега выбрано распределение $\phi_k$ над списком всевозможных слов (словарем из N слов). По сути, каждое $\phi_k$ - это вектор длины N из неотрицательных величин, в сумме дающих 1. Вектора $\phi_k$ независимы и моделируются распредеделением Дирихле $Dir(\beta)$. Теперь, чтобы собрать текст d из $n$ слов, будем действовать по следующей схеме:


* выберем распредление для тегов $\theta_d$. Вновь, $\theta_d$ - это вектор длины K из неотрицательных величин, в сумме дающих 1. Поэтому естественно брать $\theta_d \sim Dir(\theta | \alpha)$

* Для i от 1 до n:
  * выберем тег $z_i$ согласно распределению $\theta_d$
  * выберем слово $w_i$ из распределения для данного тега, т.е. $w_i \sim \phi_{z_i}$
  * добавляем слово $w_i$ в текст.

Полученная модель называется моделью LDA (Latent Dirichlet Allocation). Описанная  схема задает совместное распределение скрытых и наблюдаемых параметров по всем текстам корпуса размера M в виде:


$p(\textbf{w}, \textbf{z}, \theta, \phi | \alpha, \beta) = Dir(\theta | \alpha) Dir(\phi|\beta)Cat(\textbf{z}|\theta)Cat(\textbf{w}|\phi_z)$.

Здесь $\textbf{w}$ и $\textbf{z}$ обозначают вектора слов и тегов по всем текстам, $\theta$ - набор из $\theta_d$ для каждого документа (матрица $M\times K$), $\phi$ - набор из $\phi_k$ для каждого тега (матрица $K\times N$).


![img](https://www.researchgate.net/publication/336065245/figure/fig1/AS:807371718815752@1569503826964/Latent-Dirichlet-allocation-LDA-process-and-its-two-outputs-a-LDA-document.ppm)

Наша задача - восстановить распределение $p(\textbf{z}, \theta, \phi | \textbf{w}, \alpha, \beta)$.


Немного упростим жизнь, и поставим себе задачей восстановить распределение $p(\textbf{z} | \textbf{w}, \alpha, \beta) = \int\int p(\textbf{z}, \theta, \phi | \textbf{w}, \alpha, \beta)\textrm{d}\theta\textrm{d}\phi$.

В этот момент на помощь приходить алгоритм Gibbs Sampling. Напомним, для оценки набора парамеров $\textbf{z} = (z_1, z_2, ..., z_m)$ используется схема:

$z_i^{(t)} \sim p(z_i^{(t)}\ | \ z_1=z_1^{t}, ..., z_{i-1}=z_{i-1}^{t},
z_{i+1}=z_{i+1}^{t-1}, z_{m}=z_{m}^{t-1})$.

Условные распределения выводятся так. Сначала замечаем, что

$p(z_i|\textbf{z}_{\hat{i}}, \textbf{w}, \alpha, \beta) = \frac{p(z_i,\textbf{z}_{\hat{i}}, \textbf{w}| \alpha, \beta)}{p(\textbf{z}_{\hat{i}}, \textbf{w}| \alpha, \beta)}  = \frac{p(\textbf{z}, \textbf{w}| \alpha, \beta)}{p(\textbf{z}_{\hat{i}}, \textbf{w}_{\hat{i}}| \alpha, \beta)  p(w_i|\alpha, \beta)}$.

Здесь $\textbf{z}_{\hat{i}}$, $\textbf{w}_{\hat{i}}$ - вектора без $i$-oй копмоненты. 

Далее расписываем:

$p(\textbf{z}, \textbf{w}| \alpha, \beta) = \int\int p(\textbf{z}, \textbf{w}, \theta, \phi| \alpha, \beta)\textrm{d}\theta\textrm{d}\phi = \int\int Dir(\theta | \alpha) Dir(\phi|\beta)Cat(\textbf{z}|\theta)Cat(\textbf{w}|\phi_z)\textrm{d}\theta\textrm{d}\phi = 
\int Dir(\theta | \alpha) Cat(\textbf{z}|\theta)\textrm{d}\theta \int Dir(\phi|\beta)Cat(\textbf{w}|\phi_z)\textrm{d}\phi$

и обнаруживаем, что оба интеграла в последнем выражении вычисляются аналитически. Для примера первый:

$\int Dir(\theta | \alpha) Cat(\textbf{z}|\theta)\textrm{d}\theta = \prod\limits_d \int Dir(\theta_d | \alpha) Cat(\textbf{z}_d|\theta_d)\textrm{d}\theta_d = \prod\limits_d \int \frac{1}{B(\alpha)}\prod\limits_k \theta_{d, k}^{\alpha-1}\prod\limits_i \theta_{d, z_i}\textrm{d}\theta_d = \prod\limits_d\frac{1}{B(\alpha)}\int\prod\limits_k \theta_{d, k}^{n_{d, k} + \alpha - 1}\textrm{d}\theta_d = \prod\limits_d \frac{B(n_{d,\cdot} + \alpha)}{B(\alpha)}$.

Здесь  $n_{d,k}$ - количество тэгов $k$ в тексте $d$, $n_{d,\cdot}$ - вектор длины $K$ из этих величин.

Аналогично, второй интеграл 
$\int Dir(\phi|\beta)Cat(\textbf{w}|\phi_z)\textrm{d}\phi = \prod\limits_k \frac{B(n_{k,\cdot} + \beta)}{B(\beta)}$,

где $n_{k,\cdot}$ - вектор длины $N$ встречаемости слов внутри тэга $k$.

Получаем: 

$p(\textbf{z}, \textbf{w}| \alpha, \beta) = \prod\limits_d \frac{B(n_{d,\cdot} + \alpha)}{B(\alpha)} \prod\limits_k \frac{B(n_{k,\cdot} + \beta)}{B(\beta)}$.

Теперь
$p(z_i|\textbf{z}_{\hat{i}}, \textbf{w}, \alpha, \beta) \propto \prod\limits_d \frac{B(n_{d,\cdot} + \alpha)}{B(n_{d,\cdot}^{\hat{i}} + \alpha)} \prod\limits_k \frac{B(n_{k,\cdot} + \beta)}{B(n_{k,\cdot}^{\hat{i}} + \beta)}$.

Знак $\propto$ означает пропорциональность с точностью до общего множителя $p(w_i|\alpha, \beta)$. Векторы $n_{d,\cdot}^{\hat{i}}$ и $n_{k,\cdot}^{\hat{i}}$ получены из векторов $n_{d,\cdot}$ и $n_{k,\cdot}$ после выбрасывания $z_i$. 

Выражение упрощается дальше, расписывая бета-функцию через гамма-функции. Напомним,
$B(x_1, ..., x_m) = \frac{\Gamma(x_1)\cdot...\cdot\Gamma(x_m)}{\Gamma(x_1 + ... + x_m)}$, а также $\Gamma(n) = (n-1)\Gamma(n-1)$. Получим:

$p(z_i=k |\textbf{z}_{\hat{i}}, \textbf{w}, \alpha, \beta) \propto (n_{d_i, k}^{\hat{i}} + \alpha_k) \frac{n_{k, w_i}^{\hat{i}} + \beta_{w_i}}{\sum\limits_{w}(n_{k, w}^{\hat{i}} + \beta_{w})}$.


С этого места можно полностью собрать алгоритм моделирования плотности $p(\textbf{z}| \textbf{w}, \alpha, \beta)$. Введем  обозначение $n_k$ - количество слов, отнесенных к тегу $k$, $W$ - общее количество слов в корпусе, $\beta_{sum} = \sum\limits_w\beta_w$

Алгоритм:

* заведем счетчики $n_{k, w}$, $n_{d, k}$, $n_k$
* случайным образом расставим теги словам, обновим счетчики $n_{k, w}$, $n_{d, k}$, $n_k$
* пока не сойдемся к стационарному режиму:
  * для каждого $i$ от 1 до $W$:
      * для каждого $k$ от 1 до $K$:
        * $I = I\{z_i = k\}$ (индикатор)
        * вычисляем $p_k = (n_{d_i, k} + \alpha_k - I) \frac{n_{k, w_i} + \beta_{w_i} - I}{n_k + \beta_{sum} - I}$
      * сэмплим новый $z_i$ из полученного распределения $(p_1, ..., p_K)$
      * обновляем счетчики для учета обновленого значения $z_i$

На практике удобно реализовавать так:

* заведем счетчики $n_{k, w}$, $n_{d, k}$, $n_k$, заполненные нулями
* случайным образом расставим теги словам, обновим счетчики $n_{k, w}$, $n_{d, k}$, $n_k$
* пока не сойдемся к стационарному режиму:
  * для каждого $i$ от 1 до $W$:
      * $n_{d_i, z_i} \mathrel{-}= 1$, $n_{z_i, w_i} \mathrel{-}= 1$, $n_{z_i} \mathrel{-}= 1$
      * для каждого $k$ от 1 до $K$:
        * вычисляем $p_k = (n_{d, k} + \alpha_k) \frac{n_{k, w_i} + \beta_{w_i}}{n_k + \beta_{sum}}$
      * сэмплим новый $z_i$ из полученного распределения $(p_1, ..., p_K)$
      * $n_{d_i, z_i} \mathrel{+}= 1$, $n_{z_i, w_i} \mathrel{+}= 1$, $n_{z_i} \mathrel{+}= 1$





Восстановив распредление для $\textbf{z}$, можем оценить $\theta$ и $\phi$, о которых мы ненадолго забыли. Оценить можно, например, через матожидание по апостериорным распределениям. Получите формулы самостоятельно!

Литература:

http://u.cs.biu.ac.il/~89-680/darling-lda.pdf

https://www.cs.cmu.edu/~mgormley/courses/10701-f16/slides/lecture20-topic-models.pdf

Перейдем к практике.

## Датасет

Возьмем популярный датасет [20 Newsgroups](http://qwone.com/~jason/20Newsgroups/), встроенный в пакет ```sklearn```. Датасет состоит из ~20К текстов, классифицированных на 20 категорий. Датасет разбит на ```train``` и ```test```. Для загрузки используем  модуль ```fetch_20newsgroups```, в параметрах указать, что мета информацию о тексте загружать не нужно:

In [122]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

Выведем список категорий текстов:

In [458]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Атрибут ```traget``` хранит номера категорий для текстов из обучающей выборки:

In [124]:
newsgroups_train.target[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

Доступ к самим текстам через атрибут ```data```. Выведем текст и категорию случайного примера из обучающего датасета:

In [8]:
n = 854
print('Topic = {0}\n'.format(newsgroups_train.target_names[newsgroups_train.target[n]]))
print(newsgroups_train.data[n])

Topic = rec.motorcycles

hey... I'm pretty new to the wonderful world of motorcycles... I just
bought
a used 81 Kaw KZ650 CSR from a friend.... I was just wondering what kind of

saddle bags I could get for it (since I know nothing about them)  are there
bags for the gas tank?  how much would some cost, and how much do they
hold?
thanks for your advice!!!  I may be new to riding, but I love it
already!!!!
:)




## Векторное представление текста

Представим текст как вектор индикаторов вхождений слов из некоторого словаря в текст. Это простейшая модель BOF. 

Сформируем словарь на основе нашего набора текстов. Для этого используем модуль ```CountVectorizer```:

In [240]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,
                             analyzer='word',min_df = 0.002)
vectorizer.fit(newsgroups_train.data)

CountVectorizer(min_df=0.002,
                stop_words=frozenset({'a', 'about', 'above', 'across', 'after',
                                      'afterwards', 'again', 'against', 'all',
                                      'almost', 'alone', 'along', 'already',
                                      'also', 'although', 'always', 'am',
                                      'among', 'amongst', 'amoungst', 'amount',
                                      'an', 'and', 'another', 'any', 'anyhow',
                                      'anyone', 'anything', 'anyway',
                                      'anywhere', ...}))

Количество проиндексированных слов:

In [241]:
len(vectorizer.vocabulary_)

5358

Проиндексированные слова и их индексы:

In [363]:
vectorizer.vocabulary_

{'wondering': 5293,
 'car': 889,
 'saw': 4243,
 'day': 1388,
 'door': 1616,
 'sports': 4573,
 'looked': 2899,
 'late': 2763,
 'early': 1669,
 'called': 860,
 'doors': 1617,
 'really': 3943,
 'small': 4480,
 'addition': 320,
 'separate': 4338,
 'rest': 4103,
 'body': 752,
 'know': 2731,
 'model': 3133,
 'engine': 1758,
 'specs': 4550,
 'years': 5344,
 'production': 3779,
 'history': 2354,
 'info': 2515,
 'looking': 2900,
 'mail': 2941,
 'fair': 1920,
 'number': 3310,
 'souls': 4521,
 'upgraded': 5071,
 'si': 4415,
 'clock': 1040,
 'shared': 4385,
 'experiences': 1870,
 'send': 4328,
 'brief': 792,
 'message': 3061,
 'procedure': 3764,
 'speed': 4555,
 'cpu': 1305,
 'rated': 3913,
 'add': 317,
 'cards': 891,
 'heat': 2319,
 'hour': 2396,
 'usage': 5079,
 'floppy': 2035,
 'disk': 1570,
 'functionality': 2116,
 '800': 234,
 'floppies': 2034,
 'especially': 1797,
 'requested': 4069,
 'days': 1389,
 'network': 3260,
 'knowledge': 2733,
 'base': 644,
 'upgrade': 5070,
 'haven': 2301,
 'answer

Индекс, например, для слова anyone:

In [359]:
vectorizer.vocabulary_.get('info')

CountVectorizer(min_df=0.002,
                stop_words=frozenset({'a', 'about', 'above', 'across', 'after',
                                      'afterwards', 'again', 'against', 'all',
                                      'almost', 'alone', 'along', 'already',
                                      'also', 'although', 'always', 'am',
                                      'among', 'amongst', 'amoungst', 'amount',
                                      'an', 'and', 'another', 'any', 'anyhow',
                                      'anyone', 'anything', 'anyway',
                                      'anywhere', ...}))

А теперь преобразуем строку в вектор:

In [244]:
text = 'I was wondering if anyone out there could enlighten me on this car car I saw'
x = vectorizer.transform([text])
#x = np.array(x)
y = x.toarray()
print(y[0][25717])
#matrix_freq = np.asarray(x.sum(axis=0)).ravel()
#final_matrix = np.array([np.array(vectorizer.get_feature_names()), matrix_freq])

IndexError: index 25717 is out of bounds for axis 0 with size 5358

Какой тип имеет объект, на который указывает ```x```?

In [245]:
type(x)

scipy.sparse.csr.csr_matrix

Разреженная матрица!

### Отступление про разреженные матрицы

Список ненулевых элементов матрицы:

In [246]:
x.data

array([2, 1, 1])

Индексы строк и столбцов для ненулевых элементов:

In [247]:
x.nonzero()

(array([0, 0, 0], dtype=int32), array([ 889, 4243, 5293], dtype=int32))

Преобразование к объекту ndarray (именно после приведения к такому виду разреженные матрицы можно подставлять в функции, например, библиотеки Numpy):

In [248]:
x.toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

Вернемся к словарю. Раскодируем вектор ```x``` в список слов:

In [249]:
vectorizer.inverse_transform(x)

[array(['car', 'saw', 'wondering'], dtype='<U15')]

Пропало слово ```I```. Но дело в том, что по умолчанию ```CountVectorizer``` отбрасывает последовательности, короче 2 символов. На это указывает параметр ```token_pattern='(?u)\\b\\w\\w+\\b'```.

Переведем весь набор текстов обучающего датасета в набор векторов, получим матрицу ```X_train```:

In [250]:
X_train = vectorizer.fit_transform(newsgroups_train.data)

In [251]:
u = X_train.toarray()

In [252]:
#t = u[11314].nonzero()[0]
print(len(u))
w = []
for e in range(len(u)):
    k = []
    t = u[e].nonzero()[0]
    for i in range(len(t)):
        for j in range(u[e][t[i]]):
            k.append(t[i])
    w.append(k)

11314


In [253]:
u[11313].nonzero()

(array([  55,  144,  620,  743,  859, 1758, 2248, 2331, 2377, 2843, 2877,
        3114, 3116, 3310, 3611, 3631, 4139, 4348, 4359, 4423, 4635, 4927,
        4952, 5018, 5251]),)

In [254]:
q = np.array(w)

In [256]:
q

array([list([320, 752, 860, 889, 889, 889, 889, 1388, 1616, 1617, 1669, 1758, 2354, 2515, 2731, 2763, 2899, 2900, 2941, 3133, 3779, 3943, 4103, 4243, 4338, 4480, 4550, 4573, 5293, 5344]),
       list([234, 317, 317, 445, 644, 792, 891, 1040, 1040, 1305, 1388, 1389, 1570, 1797, 1870, 1870, 1920, 2034, 2035, 2116, 2301, 2319, 2396, 2733, 3061, 3260, 3310, 3764, 3913, 4069, 4328, 4385, 4415, 4521, 4555, 4555, 4865, 5070, 5071, 5079]),
       list([33, 61, 61, 66, 66, 66, 66, 90, 285, 306, 312, 341, 444, 450, 450, 450, 451, 704, 723, 828, 828, 1142, 1270, 1364, 1388, 1388, 1549, 1570, 1570, 1572, 1572, 1572, 1605, 1605, 1614, 1614, 1648, 1661, 1696, 1719, 1728, 1760, 1864, 1969, 1988, 1997, 1998, 2043, 2139, 2194, 2197, 2222, 2301, 2315, 2315, 2315, 2333, 2355, 2383, 2473, 2515, 2515, 2568, 2690, 2690, 2731, 2838, 2844, 2855, 2877, 2900, 2901, 2929, 2930, 2930, 2931, 2953, 2986, 3017, 3148, 3264, 3267, 3370, 3386, 3386, 3520, 3526, 3569, 3616, 3630, 3669, 3683, 3683, 3683, 3733, 3760, 3822

In [257]:
max = 0
for i in range (N_d):
    if (len(q[i]) > max):
        max = len(q[i])
max

5089

О пользе разреженных матриц. Отношение числа ненулевых элементов ко всем элементам матрицы ```X_train```:

In [144]:
X_train.nnz / np.prod(X_train.shape)

0.024243246514787098


Задача: запустить модель LDA и Gibbs Sampling с числов тегов 20. Вывести топ-10 слов по каждому тегу. Соотнести полученные теги с тегами из датасета, сделать выводы. 

In [463]:
def randomInit(ndz,nzw,nz,Z,q):
    for d, document in enumerate(q):
        сurrent = []
        for w in document:
            p = np.divide(np.multiply(ndz[d, :], nzw[:, w]), nz)
            z = np.random.multinomial(1, p / p.sum()).argmax()
            current.append(z)
            ndz[d, z] += 1
            nzw[z, w] += 1
            nz[z] += 1
        Z.append(сurrent)

In [464]:
def sampling(ndz,nzw,nz,Z,q):
    for d, document in enumerate(q):
        for i, w in enumerate(document):
            z = Z[d][i]
            ndz[d, z] -= 1
            nzw[z, w] -= 1
            nz[z] -= 1
            p = np.divide(np.multiply(ndz[d, :], nzw[:, w]), nz)
            z = np.random.multinomial(1, p / p.sum()).argmax()
            Z[d][i] = z 
            ndz[d, z] += 1
            nzw[z, w] += 1
            nz[z] += 1

In [465]:
def getKeysByValue(dictOfElements, valueToFind):
    listOfKeys = list()
    listOfItems = dictOfElements.items()
    for item  in listOfItems:
        if item[1] == valueToFind:
            listOfKeys.append(item[0])
    return  listOfKeys

In [416]:
N_w = 5358
alpha = 5
beta = 0.1
iterations = 50
Z = []
ndz = np.zeros([len(q), 20]) + alpha
nzw = np.zeros([20, N_w]) + beta
nz = np.zeros([20]) + N_w * beta

randomInit(ndz,nzw,nz,Z,q)

for i in range(0, iterations):
    sampling(ndz,nzw,nz,Z,q)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [423]:
topicwords = []
maxTopicWordsNum = 10
for z in range(0, 20):
    print(z)
    ids = nzw[z, :].argsort()
    topicword = []
    for j in ids:
        topicword.insert(0, getKeysByValue(vectorizer.vocabulary_, j))
    topicwords.append(topicword[ : min(10, len(topicword))])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [461]:
for n in range (20):
    print(n, topicwords[n])
    print('\n')
newsgroups_train.target_names

0 [['com'], ['software'], ['version'], ['image'], ['using'], ['graphics'], ['mail'], ['color'], ['display'], ['dos']]


1 [['high'], ['time'], ['new'], ['years'], ['low'], ['large'], ['earth'], ['small'], ['power'], ['point']]


2 [['good'], ['car'], ['used'], ['interested'], ['price'], ['old'], ['like'], ['sale'], ['ground'], ['don']]


3 [['just'], ['like'], ['make'], ['want'], ['think'], ['need'], ['better'], ['right'], ['big'], ['work']]


4 [['ve'], ['time'], ['going'], ['don'], ['really'], ['got'], ['long'], ['way'], ['make'], ['didn']]


5 [['new'], ['national'], ['university'], ['1993'], ['states'], ['000'], ['research'], ['american'], ['year'], ['years']]


6 [['drive'], ['card'], ['hard'], ['bit'], ['disk'], ['scsi'], ['memory'], ['speed'], ['mac'], ['video']]


7 [['problem'], ['does'], ['read'], ['case'], ['entry'], ['line'], ['sure'], ['problems'], ['time'], ['info']]


8 [['does'], ['question'], ['true'], ['right'], ['evidence'], ['case'], ['example'], ['fact'], ['argumen

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

В общем, видно, что всё неплохо. Например, для топика "атеизм" явно подходит 15-ый набор (по словам 'god','jesus'...). И так можно для многих явно понять, какие слова к ним относятся: на хокей - 16ая, на windows - 19ая. Но для некоторых (сильно смежных по смыслу) топиков, наверное, явно определить не получится, какая строка к какому из них соответствует (например, разница между 12 и 18 наборами слов лично мне особо не видна - и они оба могут подходить к чему-то из набора comp.) 